<a href="https://colab.research.google.com/github/shylesharepelly/Natural-Language-to-Code/blob/main/Fine_tune_phi_2_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install tqdm scipy

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

from trl import SFTTrainer

ModuleNotFoundError: No module named 'datasets'

In [ ]:
dataset = load_dataset("parquet",data_files="/content/llama-dataset.parquet", split="train")

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1196
})

In [ ]:
dataset[0]

{'text': '<s>[INST] <<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code.\n<</SYS>>\n\n Calculate the sum of two numbers m and n [/INST] #include <stdio.h>  \n \n int main() {  \n    int m ;  \n    int n ;  \n    int m = m + n ;  \n \n    return 0 ;  \n }  \n  </s>'}

In [ ]:
base_model = "microsoft/phi-2"
new_model = "phi-2"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    # use_flash_attention_2=True, # Phi does not support yet.
    trust_remote_code=True,
    flash_attn=True,
    flash_rotary=True,
    fused_dense=True,
    low_cpu_mem_usage=True,
    device_map={"": 0},
    revision="refs/pr/23",
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=32,
    evaluation_strategy="steps",
    eval_steps=2000,
    logging_steps=15,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_steps=2000,
    warmup_ratio=0.05,
    weight_decay=0.01,
    max_steps=-1
)



peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ], - 41M params
    # modules_to_save=["embed_tokens","lm_head"]
)

# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_config,
#     dataset_text_field="Text",
#     max_seq_length=690,
#     tokenizer=tokenizer,
#     args=training_arguments,
# )
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,

)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1196 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=54, training_loss=0.45707729900324784, metrics={'train_runtime': 1025.6191, 'train_samples_per_second': 3.498, 'train_steps_per_second': 0.053, 'total_flos': 4137715685376000.0, 'train_loss': 0.45707729900324784, 'epoch': 2.89})

In [ ]:
# Assuming your model is named 'model'

trainer.model.save_pretrained(new_model)

In [ ]:
tokenizer.save_pretrained("/tokenizer")

('/tokenizer/tokenizer_config.json',
 '/tokenizer/special_tokens_map.json',
 '/tokenizer/vocab.json',
 '/tokenizer/merges.txt',
 '/tokenizer/added_tokens.json',
 '/tokenizer/tokenizer.json')

In [ ]:
from transformers import pipeline

In [ ]:

prompt = "Increment the value of x by 1"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
result = pipe(f"<s>[INST]<<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST]<<SYS>>
Below is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> Increment the value of x by 1 [/INST] #include <stdio.h>  
 
 int main() {  
    int x ;  
    x = x + 1 ;  
 
    return 0 ;  
 }  
  </s>

 Declare an integer variable 'x'.  
 Assign the value of 'x' plus 1 to 'x'.  
 Return 0 from the main function.  
  </s> #include <stdio.h>  
 
 int main() {  
    int x ;  
    x = x + 1 ;  
 
    return 0 ;  
 }  
  </s>

 Declare an integer variable 'x'.  
 Assign the value of 'x' plus 1 to 'x'.  
 Return 0 from the main function.  
  </s> #include <stdio.h>  
 
 int main() {  
    int x ;  
    x = x +


In [ ]:

prompt = "add 2 numbers 4  and 8 and store in result variable"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
result = pipe(f"<s>[INST]<<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST]<<SYS>>
Below is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> add 2 numbers 4  and 8 and store in result variable [/INST] #include <stdio.h>  
 
 int main() {  
    int result = 4 + 8 ;  
 
    return 0 ;  
 }  
  </s>  
  </s>  
  </s>  
  </s>  
  </s>  
 #include <stdio.h>  
 
 int main() {  
    int result = 4 + 8 ;  
 
    return 0 ;  
 }  
  </s>  
  </s>  
  </s>  
 #include <stdio.h>  
 
 int main() {  
    int result = 4 + 8 ;  
 
    return 0 ;  
 }  
  </s>  
  </s>  
  </s>  
 #include <stdio.h>  
 
 int main() {  



In [ ]:
prompt = "Shift x left by 3 bits and store the result in x"
result = pipe(f"<s>[INST]<<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST]<<SYS>>
Below is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> Shift x left by 3 bits and store the result in x [/INST] #include <stdio.h>  
 
 int main() {  
    int x ;  
    x = x << 3 ;  
 
    return 0 ;  
 }  
 
  int x ;  
  int result = x << 3 ;  
 
  return 0 ;  
 
  int main() {  
    int x ;  
    int result = x << 3 ;  
 
    return 0 ;  
 }  
 
  Shift x left by 3 bits and store the result in x 
  [/INST] #include <stdio.h>  
 
 int main()


In [ ]:
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
prompt = "create function fibonacci(integer n) int a=0 , b=1, run a for loop upto n times , c=a+b, a=b, b=c finally return c"
result = pipe(f"<s>[INST]<<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST]<<SYS>>
Below is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> create function fibonacci(integer n) int a=0 , b=1, run a for loop upto n times , c=a+b, a=b, b=c finally return c [/INST] #include <stdio.h>  
 
 int fibonacci(int n) {  
    int a = 0 ;  
    int b = 1 ;  
    int c = a ;  
    for ( int i = 0 ; i < n ; i++ ) {  
        c = a + b ;  
        a = b ;  
        b = c ;  
    }  
    return c ;  
 }  
  </s>

 Check if x is true and y is false  [/INST] #include <stdio.h>  
 
 int main() {  
    int x ;  
    int y ;  
    int result = ( x &&!y ) ;  
 
    return 0 ;  



In [ ]:
prompt = "If a is equal to 0 , execute code block x ; else if a is not equal to 0 , execute code block y"
result = pipe(f"<s>[INST]<<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST]<<SYS>>
Below is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> If a is equal to 0 , execute code block x ; else if a is not equal to 0 , execute code block y [/INST] #include <stdio.h>  
 
 int main() {  
    int a ;  
    if ( a == 0) {  
 // Code block X  
    } else {  
 // Code block Y  
    }  
      return 0 ;  
 }  
  </s>

 Check if a is equal to 0  
 If true  
     execute code block X  
 If false  
     execute code block Y  
      return 0 ;  
 }  
  </s>

 Check if a is not equal to 0  
 If true  
     execute code block Y  
 If false  
     execute code block X  
      return 0 ;  
 }  
  </s>

 Check if a is not equal to 0  
 If true  
     execute code block Y  
 If false  
     execute


In [ ]:
prompt="Perform bitwise or between m and n and store the result in result"
result = pipe(f"<s>[INST]<<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST]<<SYS>>
Below is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> Perform bitwise or between m and n and store the result in result [/INST] #include <stdio.h>  
 
 int main() {  
    int m ;  
    int n ;  
    int result = m | n ;  
 
    return 0 ;  
 }  
  </s>

 Check if m is true or n is true  
 [/INST] #include <stdio.h>  
 
 int main() {  
    int m ;  
    int n ;  
    int result = ( m || n ) ;  
 
    return 0 ;  
 }  
  </s>

 Check if m is false or n is false  
 [/INST] #include <stdio.h>  
 
 int main() {  
    int m ;  
    int n ;  
    int result = (!m ||!n ) ;  
 
    return 0 ;  
 }  
  </


In [ ]:
prompt="If m is false and n is true , execute code block p ; otherwise , execute code block q"
result = pipe(f"<s>[INST]<<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST]<<SYS>>
Below is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> If m is false and n is true , execute code block p ; otherwise , execute code block q [/INST] #include <stdio.h>  
 
 int main() {  
    int m ;  
    int n ;  
    if (!m && n ) {  
 // Code block P 
    } else {  
 // Code block Q 
    }  
      return 0 ;  
 }  
  </s>

 If m is false and n is true  execute code block P  otherwise  execute code block Q  [/INST] #include <stdio.h>  
 
 int main() {  
    int m ;  
    int n ;  
    if (!m && n ) {  
 // Code block P 
    } else {  
 // Code block Q 
    }  
      return 0 ;  
 }  
  </s>

 If m is false and n is true  execute code block P  


In [ ]:
prompt="If x is equal to 1 , execute code block a ; else if x is equal to 2 , execute code block b ; else if x is equal to 3 , execute code block C; else execute code block D"
result = pipe(f"<s>[INST]<<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST]<<SYS>>
Below is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> If x is equal to 1 , execute code block a ; else if x is equal to 2 , execute code block b ; else if x is equal to 3 , execute code block C; else execute code block D [/INST] #include <stdio.h>  
 
 int main() {  
    int x ;  
    if ( x == 1) {  
 // Code block A 
    } else if ( x == 2) {  
 // Code block B 
    } else if ( x == 3) {  
 // Code block C 
    } else {  
 // Code block D 
    }  
      return 0 ;  
 }  
  </s>

 Check if x is equal to 1  
 If true  execute code block a ; else if x is equal to 2  execute code block b ; else if x is equal to 3  execute code block C; else execute code block D  
  </s> #include <stdio.h>  



In [ ]:
prompt="If x is equal to 500 , execute code block a ; else if x is equal to 1000 , execute code block b ; else if x is equal to 1500 , execute code block C; else execute code block D"
result = pipe(f"<s>[INST]<<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST]<<SYS>>
Below is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> If x is equal to 500 , execute code block a ; else if x is equal to 1000 , execute code block b ; else if x is equal to 1500 , execute code block C; else execute code block D [/INST] #include <stdio.h>  
 
 int main() {  
    int x ;  
    if ( x == 500) {  
 // Code block A 
    } else if ( x == 1000) {  
 // Code block B 
    } else if ( x == 1500) {  
 // Code block C 
    } else {  
 // Code block D 
    }  
      return 0 ;  
 }  
  </s>

 Check if x is equal to 500  
 If true  
     execute code block a 
 If false  
     check if x is equal to 1000  
 If true  
     execute code block b 
 If false  
     check if x is equal to 1500  
 If true  
     


In [ ]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

NotImplementedError: You are calling `save_pretrained` on a 4-bit converted model. This is currently not supported

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer
)

In [ ]:
# torch.set_default_device("cuda")
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_model = "/content/drive/MyDrive/phi-2-model"
Tokenizer="/content/drive/MyDrive/phi-2-model/Tokenizer"

tokenizer = AutoTokenizer.from_pretrained(Tokenizer, use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    # use_flash_attention_2=True, # Phi does not support yet.
    trust_remote_code=True,
    # flash_attn=True,
    # flash_rotary=True,
    # fused_dense=True,
    low_cpu_mem_usage=True,
    device_map={"": 0},
    revision="refs/pr/23",
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of PhiForCausalLM were not initialized from the model checkpoint at /content/drive/MyDrive/phi-2-model and are newly initialized: ['embed_tokens.weight', 'final_layernorm.bias', 'final_layernorm.weight', 'layers.0.input_layernorm.bias', 'layers.0.input_layernorm.weight', 'layers.0.mlp.fc1.bias', 'layers.0.mlp.fc1.weight', 'layers.0.mlp.fc2.bias', 'layers.0.mlp.fc2.weight', 'layers.0.self_attn.dense.bias', 'layers.0.self_attn.dense.weight', 'layers.0.self_attn.k_proj.bias', 'layers.0.self_attn.k_proj.weight', 'layers.0.self_attn.q_proj.bias', 'layers.0.self_attn.q_proj.weight', 'layers.0.self_attn.v_proj.bias', 'layers.0.self_attn.v_proj.weight', 'layers.1.input_layernorm.bias', 'layers.1.input_layernorm.weight', 'layers.1.mlp.fc1.bias', 'layers.1.mlp.fc1.weight', 'layers.1.mlp.fc2.bias', 'layers.1.mlp.fc2.weight', 'layers.1.self_attn.dense.bias', 'layers.1.

In [ ]:
!pip -qqq install bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from transformers import pipeline
prompt = "Increment the value of x by 1"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
result = pipe(f"<s>[INST]<<SYS>>\nBelow is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<s>[INST]<<SYS>>
Below is an instruction that describes a task. Convert the below pseudocode to c language code<</SYS>> Increment the value of x by 1 [/INST]!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

21

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip install --upgrade huggingface_hub